Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Data Driven Audio Signal Processing - A Tutorial with Computational Examples

Winter Semester 2021/22 (Master Course #24512)

- lecture: https://github.com/spatialaudio/data-driven-audio-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Exercise 4: SVD and Right Matrix Inverse

## Objectives

- SVD for flat/fat, full row rank matrix
- Four subspaces in SVD domain
- Projection matrices
- Right inverse

## Special Python Packages

Some convenient functions are found in `scipy.linalg`, some in `numpy.linalg` 

## Some Initial Python Stuff

In [60]:
import numpy as np
from scipy.linalg import svd, diagsvd, inv, pinv, null_space, norm
from numpy.linalg import matrix_rank

np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

rng = np.random.default_rng(1234)
mean, stdev = 0, 1

# we might try out that this all works for complex data, complex vector spaces as well
# for complex data ^H operator (conj().T) needs used instead of transpose only!!!
use_complex = False

## SVD of Matrix A

In [61]:
M = 3  # number of rows
N = 7  # number of cols

rank = min(M, N)  # set desired rank == full column rank == independent columns
print('desired rank of A:', rank)

if use_complex:
    dtype = 'complex128'
    A = np.zeros([M, N], dtype=dtype)
    for i in range(rank):
        col = rng.normal(mean, stdev, M) + 1j*rng.normal(mean, stdev, M)
        row = rng.normal(mean, stdev, N) + 1j*rng.normal(mean, stdev, N)
        A += np.outer(col, row)
else:
    dtype = 'float64'
    A = np.zeros([M, N], dtype=dtype)
    for i in range(rank):
        col = rng.normal(mean, stdev, M)
        row = rng.normal(mean, stdev, N)
        A += np.outer(col, row)
# check if rng produced desired rank
print('        rank of A:', matrix_rank(A))
print('flat/fat matrix with full row rank')
print('-> matrix U contains only the column space')
print('-> left null space is only the zero vector')
print('A =\n', A)

desired rank of A: 3
        rank of A: 3
flat/fat matrix with full row rank
-> matrix U contains only the column space
-> left null space is only the zero vector
A =
 [[-0.82 -0.55 -3.47  2.14 -2.40  2.22 -0.20]
 [-0.19 -1.09 -2.40  0.46  2.37  0.06 -1.78]
 [-1.71  2.56  4.43 -1.51 -0.77 -2.49  0.38]]


In [62]:
[U, s, Vh] = svd(A)
S = diagsvd(s, M, N)
V = Vh.conj().T

print('U =\n', U)
# number of non-zero sing values along diag must match rank
print('S =\n', S)
print('V =\n', V)
print('non-zero singular values: ', s[:rank])

U =
 [[-0.56  0.71  0.43]
 [-0.33 -0.66  0.67]
 [ 0.76  0.23  0.60]]
S =
 [[7.87 0.00 0.00 0.00 0.00 0.00 0.00]
 [0.00 4.06 0.00 0.00 0.00 0.00 0.00]
 [0.00 0.00 2.07 0.00 0.00 0.00 0.00]]
V =
 [[-0.10 -0.21 -0.73  0.22 -0.03 -0.22 -0.56]
 [ 0.33  0.23  0.28  0.04  0.71 -0.10 -0.49]
 [ 0.78  0.04 -0.20  0.31 -0.21  0.46  0.05]
 [-0.32  0.22  0.15  0.90  0.06 -0.00  0.13]
 [-0.00 -0.85  0.05  0.13  0.42  0.17  0.22]
 [-0.40  0.24 -0.25 -0.16  0.27  0.78 -0.08]
 [ 0.13  0.28 -0.51 -0.06  0.44 -0.29  0.61]]
non-zero singular values:  [7.87 4.06 2.07]


## Four Subspaces in SVD Domain



TBD:
The null space $N(\mathbf{A})$ of the tall/thin, full column rank matrix $\mathbf{A}$ is only $\mathbf{0}$, i.e. $N(\mathbf{A})=\mathbf{0}$. Except for $\mathbf{x}=\mathbf{0}$ all other $\mathbf{x}$ are mapped to the column space $C(\mathbf{A})$. This, however, requires, that the $\mathbf{V}$ matrix completely spans the row space and no $\mathbf{v}$ vectors span a dedicated null space.

The tall/thin, full column rank matrix $\mathbf{A}$ spans a rather large left null space $N(\mathbf{A}^\mathrm{H})$ with dimension $M-\mathrm{rank}(A)$.

We, therefore, here deal with a linear set of equations with **more equations than unknowns** ($M>N$, more rows than columns) , i.e. the **over-determined** case. For this case, we can find a solution in **least-sqaures** sense by help of the **left inverse** as discussed below.

In [63]:
# all stuff that is in matrix U
print('U =\n', U)

# Column Space C(A)
print('\ncolumn space (orthogonal to left null space):')
print(U[:, :rank])

# Left Null Space, if empty only 0 vector
print('left null space (orthogonal to column space):')
print(U[:, rank:])  # for full row rank this is only the zero vector

print('###')

# all stuff that is in matrix V
print('\nV =\n', V)

# Row Space
print('\nrow space (orthogonal to null space):')
print(V[:, :rank])

# Null Space N(A), if empty only 0 vector
print('null space (orthogonal to row space):')
print(V[:, rank:])

U =
 [[-0.56  0.71  0.43]
 [-0.33 -0.66  0.67]
 [ 0.76  0.23  0.60]]

column space (orthogonal to left null space):
[[-0.56  0.71  0.43]
 [-0.33 -0.66  0.67]
 [ 0.76  0.23  0.60]]
left null space (orthogonal to column space):
[]
###

V =
 [[-0.10 -0.21 -0.73  0.22 -0.03 -0.22 -0.56]
 [ 0.33  0.23  0.28  0.04  0.71 -0.10 -0.49]
 [ 0.78  0.04 -0.20  0.31 -0.21  0.46  0.05]
 [-0.32  0.22  0.15  0.90  0.06 -0.00  0.13]
 [-0.00 -0.85  0.05  0.13  0.42  0.17  0.22]
 [-0.40  0.24 -0.25 -0.16  0.27  0.78 -0.08]
 [ 0.13  0.28 -0.51 -0.06  0.44 -0.29  0.61]]

row space (orthogonal to null space):
[[-0.10 -0.21 -0.73]
 [ 0.33  0.23  0.28]
 [ 0.78  0.04 -0.20]
 [-0.32  0.22  0.15]
 [-0.00 -0.85  0.05]
 [-0.40  0.24 -0.25]
 [ 0.13  0.28 -0.51]]
null space (orthogonal to row space):
[[ 0.22 -0.03 -0.22 -0.56]
 [ 0.04  0.71 -0.10 -0.49]
 [ 0.31 -0.21  0.46  0.05]
 [ 0.90  0.06 -0.00  0.13]
 [ 0.13  0.42  0.17  0.22]
 [-0.16  0.27  0.78 -0.08]
 [-0.06  0.44 -0.29  0.61]]


## Right Inverse via SVD

In [64]:
Si = diagsvd(1/s, N, M)  # works if array s has only non-zero entries
print('Inverse singular value matrix with lower zero block')
print('Si =\n', Si)
# right inverse using 'inverse' SVD:
Ari = V @ Si @ U.conj().T
# right inverse using a dedicated pinv algorithm
# proper choice is done by pinv() itself
Ari_pinv = pinv(A)
print('pinv() == inverse SVD?', np.allclose(Ari, Ari_pinv))
print('S @ Si = \n', S @ Si, '\nyields MxM identity matrix')
print('A @ Ari = \n', A @ Ari, '\nyields MxM identity matrix')

Inverse singular value matrix with lower zero block
Si =
 [[0.13 0.00 0.00]
 [0.00 0.25 0.00]
 [0.00 0.00 0.48]
 [0.00 0.00 0.00]
 [0.00 0.00 0.00]
 [0.00 0.00 0.00]
 [0.00 0.00 0.00]]
pinv() == inverse SVD? True
S @ Si = 
 [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]] 
yields MxM identity matrix
A @ Ari = 
 [[ 1.00 -0.00 -0.00]
 [-0.00  1.00 -0.00]
 [-0.00 -0.00  1.00]] 
yields MxM identity matrix


## Projection Matrices for the Left Inverse Problem

In [65]:
v_row = V[:,0]
v_null = V[:,rank]
v_tmp = v_row + v_null

# projection onto row space
P_CAH = Ari @ A
print('P_CAH projects V-space stuff to row space:\n', P_CAH @ (v_tmp), '==', v_row)

# projection onto column space
# full rank and identity since we dont have a left null space
# so each column space vector is projected onto itself
P_CA = A @ Ari  #  = I_MxM
print('P_CA is projection a column space vector onto itself:\n', P_CA @ U[:,0], '==', U[:,0])

# projection onto null space, for flat/fat this space might be very large
P_NA = np.eye(N,N) - P_CAH
print('P_NA projects V-space stuff to null space:\n', P_NA @ (v_tmp), '==', v_null)

# projection onto left null space
P_NAH = np.eye(M,M) - P_CA  # == null matrix

P_CAH projects V-space stuff to row space:
 [-0.10  0.33  0.78 -0.32 -0.00 -0.40  0.13] == [-0.10  0.33  0.78 -0.32 -0.00 -0.40  0.13]
P_CA is projection a column space vector onto itself:
 [-0.56 -0.33  0.76] == [-0.56 -0.33  0.76]
P_NA projects V-space stuff to null space:
 [ 0.22  0.04  0.31  0.90  0.13 -0.16 -0.06] == [ 0.22  0.04  0.31  0.90  0.13 -0.16 -0.06]


## Copyright

- the notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources)
- feel free to use the notebooks for your own purposes
- the text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
- the code of the IPython examples is licensed under under the [MIT license](https://opensource.org/licenses/MIT)
- please attribute the work as follows: *Frank Schultz, Data Driven Audio Signal Processing - A Tutorial Featuring Computational Examples, University of Rostock* ideally with relevant file(s), github URL https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise, commit number and/or version tag, year.
